In [16]:
from pymongo import MongoClient
import datetime
import calendar
import pandas as pd
import numpy as np
import Config
import time
pd.set_option('display.max_rows', None)
client = MongoClient(Config.DB_Hostname,Config.DB_Port)

def LastThInMonth(year, month):
    
    # Create a datetime.date for the last day of the given month
    daysInMonth = calendar.monthrange(year, month)[1]   # Returns (month, numberOfDaysInMonth)
    dt = datetime.date(year, month, daysInMonth)

    # Back up to the most recent Thursday
    offset = 4 - dt.isoweekday()
    if offset > 0: offset -= 7                          # Back up one week if necessary
    dt += datetime.timedelta(offset)                    # dt is now date of last Th in month
    
    return dt

def _days_to_expiry(underlying, start_date,end_date):

    #Beware of 3 or more consequtive holidays
    start_date=datetime.datetime.strptime(start_date,"%Y-%m-%d")
    end_date=datetime.datetime.strptime(end_date,"%Y-%m-%d")

    holidays=pd.DataFrame(client.Strategy.Holidays.find()).sort_values('date')
    holidays.drop(columns=["_id"],inplace=True)

    date=start_date
    date_string=[]
    current_week=[]
    next_week=[]

    while date != end_date:
        w=date.weekday()
        if w not in [5,6]:
            date_string.append(datetime.datetime.strftime(date,"%Y-%m-%d"))
        date+=datetime.timedelta(days = 1)

    extra_days=pd.DataFrame(client.Strategy.Working_Holidays.find()).sort_values('date')

    for date in extra_days.date:
        date_string.append(date)

    date_string=sorted(date_string)


    df=pd.DataFrame()
    df["date"]=date_string
    df["Holiday"]=np.where(df.date.isin(holidays.date),1,0)
    df["week_day"]=df.date.apply(lambda x : datetime.datetime.strptime(x,"%Y-%m-%d").weekday())

    if underlying == "NIFTY":
        df["expiry"]=np.where(df.week_day==3,1,0)
    else:
        monthly_expiry = []
        for year in list(range(2015, end_date.year + 1)):
            for month in range(1, 13):
                monthly_expiry.append(datetime.datetime.strftime(LastThInMonth(year, month), "%Y-%m-%d"))

        faulty_wednesday_expiry = [datetime.datetime.strftime(datetime.datetime.strptime(thursday_expiry, "%Y-%m-%d") - datetime.timedelta(days = 1), "%Y-%m-%d") for thursday_expiry in monthly_expiry]
        df["expiry"]=np.where(df.week_day==2,1,0)
        df.expiry = np.where(df.date.isin(faulty_wednesday_expiry), 0, df.expiry)
        df.expiry = np.where(df.date.isin(monthly_expiry), 1, df.expiry)

    week_day=df.week_day.tolist()
    Holiday=df.Holiday.tolist()
    date_string=df.date.tolist()
    normal_expiry=df.expiry.tolist()
    expiry_day=[]

    for i,date in enumerate(date_string):
        if normal_expiry[i]==1:
            if Holiday[i]==0:
                expiry_day.append(date)
            elif Holiday[i-1]==0:
                expiry_day.append(date_string[i-1])
            else:
                expiry_day.append(date_string[i-2])
    df["actual_expiry"]=np.where(df.date.isin(expiry_day),1,0)
    expiry_day=df.actual_expiry.tolist()

    current_week=[]
    next_week=[]

    for i,date in enumerate(date_string):

        if Holiday[i]==1:
            current_week.append(-1)
            next_week.append(-1)
            continue

        expiry_count=0
        curr_temp=0
        next_temp=0

        try:
            for j,date_ in enumerate(date_string[i:]):

                if Holiday[i+j]==1:
                    continue
                
                next_temp = date_
                if expiry_count<1:
                    curr_temp = date_
                    
                #next_temp+=1        
                #if expiry_count<1:
                #    curr_temp+=1

                if expiry_day[i+j]==1:
                    expiry_count+=1

                if expiry_count==2:
                    break
        except Exception as e:
            print(e)
            current_week.append(np.nan)
            next_week.append(np.nan)
        
        current_week.append((datetime.datetime.strptime(curr_temp, "%Y-%m-%d").date() - datetime.datetime.strptime(date, "%Y-%m-%d").date()).days + 1)
        next_week.append((datetime.datetime.strptime(next_temp, "%Y-%m-%d").date() - datetime.datetime.strptime(date, "%Y-%m-%d").date()).days + 1)


    df["current_week"]=current_week
    df["next_week"]=next_week

    df=df[df.Holiday==0]
    df=df[['date','current_week','next_week']]
    df.sort_values('date',inplace=True)
    df["underlying"]=underlying
    return df

def Days_To_Expiry(start_date, end_date):
    
    Nifty_df = _days_to_expiry("NIFTY", start_date, end_date)
    Banknifty_df = _days_to_expiry("BANKNIFTY", start_date, end_date)
    Banknifty_df = pd.concat([Nifty_df[Nifty_df.date <= "2023-09-01"], Banknifty_df[Banknifty_df.date > "2023-09-01"]])
    Banknifty_df.underlying = "BANKNIFTY"
    
    df = pd.concat([Nifty_df, Banknifty_df]).sort_values(['date', 'underlying'])
    
    print(client.Strategy.Days_To_Expiry.delete_many({}).deleted_count)
    print(len(client.Strategy.Days_To_Expiry.insert_many(df.to_dict('records')).inserted_ids))

In [17]:
df = Days_To_Expiry("2015-01-08", "2024-12-31")

4920
4922
